0) Just some imports

In [443]:
import re
import math
import numpy as np
import common as cm
from scipy import spatial

# Query Expansion

# 1) Simple search engine (for 3.0)

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [444]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.idf_dict = {}
        
        self.corM = [] ### a correlation matrix
        self.corM_A = [] ### helpful matrices
        self.corM_transposed = []
        self.corM_final = []
        
        self.idfs_dict = {}

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### Version with words derived from documents
    ### IDF = log(N / df(w)); N - total number of documents, df(w) - frequency of documents containing word 'w'
    def computeIDFs(self, documents):
        
        self.idfs = []
        N = len(documents)
        
        self.idf_dict = {}
        for document in documents:
            
            # check which tokens are in document
            idf_dict = {}
            for token in document.tokens:
                if token not in idf_dict:
                    idf_dict[token] = 1
                    
            # count number of documents for each term
            for key, value in idf_dict.items():
                if key not in self.idf_dict:
                    self.idf_dict[key] = 1
                else:
                    self.idf_dict[key] += 1

        for value in self.idf_dict.values():
            self.idfs.append(math.log10(N / value))
            
    def computeCorM(self, documents):
        
        # TODO: terms are only those which are in dictionary (?)
        # COMPUTE A
        blank_row = [0]*len(documents)
        row_term_id_dict = {} 
        # for iter, term in enumerate(self.terms):
        #    row_term_id_dict[term] = iter
        #    self.corM_A.append(blank_row)
        
        # TODO: terms are only those which are in documents
        i = 0
        for document in documents:
            for token in document.tokens:
                if token not in row_term_id_dict:
                    row_term_id_dict[token] = i
                    self.corM_A.append(blank_row)
                    i += 1
        print(row_term_id_dict)
            
        # COMPUTE A -> look for term in all documents, if found, set -1 in a cell
        for document in documents:
            tokens = document.tokens
            update_count = 0
            for token in tokens:
                if token in row_term_id_dict:
                    self.corM_A[row_term_id_dict[token]][document.doc_id] += 1
                    update_count += 1
                else: 
                    print("Token not found in dictionary")
            
        # COMPUTE A-NORM -> TODO: divide by number of occurences in row (?)
        for i, row in enumerate(self.corM_A):
            sum = 0
            for el in row:
                sum += el
            
            for j in range(len(row)):
                self.corM_A[i][j] /= sum
        
        # COMPUTE A-NORM-TRANSPOSE
        for j in range(len(self.corM_A[0])):
            row = []
            for i in range(len(self.corM_A[0])):
                row.append(self.corM_A[i][j])
            self.corM_transposed.append(row)
            
        # COMPUTE CORRELATION MATRIX
        self.corM = np.dot(np.array(self.corM_A),np.array(self.corM_transposed))
                     
### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [445]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [446]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [447]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation
        
        self.bow_lenght = 0
        self.bow_dict = {}

    ### complete this method; it should compute a BOW representation
    ### text is represented as multisets of its words
    def computeBOW_Representation(self):
        self.bow = []
        bow_dict = {}
        for token in self.tokens:
            if token not in bow_dict:
                bow_dict[token] = 1
                self.bow_lenght += 1
            else:
                bow_dict[token] += 1
                #self.bow.append(0) # fill in the tab
        self.bow = bow_dict.items()
        
        
    ### complete this method; it should compute a TF representation
    ### el: (number of times t occurs in d / total number of terms in d)
    def computeTF_Representation(self):
        self.tf = []
        denominator = self.bow_lenght
        for term in self.bow:
            self.tf.append(term[1] / denominator)
        
    ### complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    ### TFxIDF(t,d,D) = TF(t,d) * IDF(t,D)
    def computeTF_IDF_Representation(self):
        self.tf_idf = []
        for tf, key in zip(self.tf, self.bow_dict.keys()):
            idf = self.dictionary.idf_dict[key]
            print("idf: ", idf)
            self.tf_idf.append(tf * idf)
        
        
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]


1.4) Compute IDFs here

In [458]:
### COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])

# LEAST COMMON WORDS - HIGH IDF
print("Least common words: ", res[-5:], "\n")

# MOST COMMON WORDS - LOW IDF
print("Most common words: ", res[:5], "\n")

Least common words:  [['yahoo', 1.9444826721501687], ['york', 1.9444826721501687], ['you', 1.9444826721501687], ['young', 1.9444826721501687], ['zdm', 1.9444826721501687]] 

Most common words:  [['academic', 0.009984220906600923], ['access', 0.009984220906600923], ['autonomous', 0.1962946451439682], ['apply', 0.3212333817522682], ['all', 0.3881801713828814]] 



1.5) Compute the document representations (for each document run computeRepresentations())

In [449]:
for d in documents: 
    d.computeRepresentations()
    
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow)

dict_items([('david', 1), ('aha', 1), ('machine', 4), ('learning', 4), ('page', 1), ('resource', 2), ('suggestion', 1), ('welcome', 1), ('wizrule', 1), ('zdm', 1), ('scientific', 1), ('ltd', 1), ('conference', 1), ('announcement', 1), ('course', 1), ('data', 1), ('repository', 1), ('description', 1), ('comprehensive', 1), ('from', 1), ('application', 1), ('tutorials', 1)])


1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [450]:
def getSimilarity(v1, v2):
    return 1 - spatial.distance.cosine(v1, v2)

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [451]:
query = "machine learning"
#query = "academic research"
#query = "international conference"
#query = "international conference washington"

In [452]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 1 | DOC ID = 0
David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.


RANK = 2 WITH SIMILARITY = 1 | DOC ID = 1
Home Pages of ML & CBR Folks
 Machine Learning and Case-Based Reasoning Home Pages. All 834 entries | Dedicated
 interface | Geographic location (Please send edits to David W. Aha). ... 
 Description: A list of home pages for people in machine learning and case-based reasoning.


RANK = 3 WITH SIMILARITY = 1 | DOC ID = 2
UCI Machine Learning Repository
 Welcome to the UCI Machine Learning Repository! ... The majority of the entries in the
 repository were contributed by machine learning researchers outside of UCI. ... 
 Description: A repository of databases, domain theories and data generators that are used by the ma

/home/kasia/.local/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/kasia/.local/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# 2) Query expansion

## 2.1) Correlation matrix (for 3.5)

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [453]:
### COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
for row in dictionary.corM:
    print(row, end="\n")

{'david': 0, 'aha': 1, 'machine': 2, 'learning': 3, 'page': 4, 'resource': 5, 'suggestion': 6, 'welcome': 7, 'wizrule': 8, 'zdm': 9, 'scientific': 10, 'ltd': 11, 'conference': 12, 'announcement': 13, 'course': 14, 'data': 15, 'repository': 16, 'description': 17, 'comprehensive': 18, 'from': 19, 'application': 20, 'tutorials': 21, 'home': 22, 'cbr': 23, 'folks': 24, 'and': 25, 'casebased': 26, 'reasoning': 27, 'all': 28, 'entry': 29, 'dedicated': 30, 'interface': 31, 'geographic': 32, 'location': 33, 'please': 34, 'send': 35, 'edit': 36, 'list': 37, 'for': 38, 'people': 39, 'uci': 40, 'the': 41, 'majority': 42, 'were': 43, 'contributed': 44, 'researcher': 45, 'outside': 46, 'database': 47, 'domain': 48, 'theory': 49, 'generator': 50, 'that': 51, 'are': 52, 'used': 53, 'group': 54, 'investigate': 55, 'mechanism': 56, 'which': 57, 'knowledge': 58, 'acquired': 59, 'through': 60, 'experience': 61, 'information': 62, 'mlnet': 63, 'ois': 64, 'find': 65, 'network': 66, 'online': 67, 'service':

2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [454]:
query = "machine"
#query = "algorithm"
# query = "learning"
# query = "conference"
# query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    print("Suggestions")
    pass
        
suggestKeywords(query, dictionary)

Suggestions


# 2.2) Rocchio algorithm (for 4.0)

- a method for updating a query vector - vector space model must be used to represent documents (TF-IDF vectors)

- linear combination of:
    * q
    * D_r
    * D_nr
    
- new q_m: 
    * alfa, beta, gamma are weights
    * beta (positive feedback) > gamma (negative feedback)
    * user assessed many documents? -> beta, gamma should be high

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) 
- Firstly, run the below code.  
- Observe the results.  
- Assume that we do not like the first and the second result (Docs 63 and 77).  
- However, assume that docs 29 and 36 are satisfactory. Now, modfify the method.  
- It should alter the query vector, according to Rocchio algorithm.   
- Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77).   
- Check the results for different values of alpha, beta, and gamma coefficients.  

In [455]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    print(ranks)
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

[[<__main__.Document object at 0x7f5bc73a0f40>, 1], [<__main__.Document object at 0x7f5bc73a0df0>, 1], [<__main__.Document object at 0x7f5bc73a0040>, 1], [<__main__.Document object at 0x7f5bc73a0070>, 1], [<__main__.Document object at 0x7f5bc73a00d0>, 1], [<__main__.Document object at 0x7f5bc73a0100>, 1], [<__main__.Document object at 0x7f5bc73a0160>, 1], [<__main__.Document object at 0x7f5bc73a0190>, 1], [<__main__.Document object at 0x7f5bc73a01c0>, 1], [<__main__.Document object at 0x7f5bc73878e0>, 1], [<__main__.Document object at 0x7f5bc73878b0>, 1], [<__main__.Document object at 0x7f5bc7387880>, 1], [<__main__.Document object at 0x7f5bc7387850>, 1], [<__main__.Document object at 0x7f5bc7387820>, 1], [<__main__.Document object at 0x7f5bc73877f0>, 1], [<__main__.Document object at 0x7f5bc73877c0>, 1], [<__main__.Document object at 0x7f5bc7387790>, 1], [<__main__.Document object at 0x7f5bc7387760>, 1], [<__main__.Document object at 0x7f5bc7387730>, 1], [<__main__.Document object at 

# 2.3) WordNet (for 5.0)

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [456]:
from nltk.corpus import wordnet as wn

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [457]:
wn.synsets('machine')

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/home/kasia/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


2.3.3) Display all definitions (.definition()) for synsets (machine)

In [ ]:
#TODO

2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [ ]:
#TODO

See: http://www.nltk.org/howto/wordnet.html
for more examples